In [1]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import nn, optim
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import os
import readligo as rl

In [2]:
epochs = 50
test_sample_ratio = .2
input_vector_length = 1024
batch_size = 200
num_bins = 40

input_dir = "/Volumes/Research/GWNMMAD_data/Downsampled/V1/"
input_file_list = ["H-H1_GWOSC_O2_4KHZ_R1-1174978560-4096_downsampled", "L-L1_GWOSC_O2_4KHZ_R1-1187291136-4096_downsampled", "V-V1_GWOSC_O2_4KHZ_R1-1187688448-4096_downsampled"]

input_file = input_file_list[1]

output_dir  = "/Volumes/Research/GWNMMAD_data/Model/"

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(1024, 768),
            nn.ReLU(),
            nn.Linear(768, 384),
            nn.ReLU(),

        )
        self.decoder = nn.Sequential(
            nn.Linear(384, 768),
            nn.ReLU(),
            nn.Linear(768, 1024),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
def bunch_normalize(v_set):
    recover_min = list(range(len(v_set)))
    recover_coef = list(range(len(v_set)))
    for i in range(len(v_set)):
        v_set[i], recover_min[i], recover_coef[i] = min_max_normalization(v_set[i])
    return v_set, recover_min, recover_coef